#Downgrade Tensorflow and Keras

In [ ]:
import tensorflow
print(tensorflow.__version__)

2.3.0


In [ ]:
!pip uninstall keras-nightly

!pip uninstall -y tensorflow

!pip install h5py==2.10.0  


In [ ]:
!pip uninstall tensorflow
!pip uninstall keras

In [ ]:
!pip install tensorflow==2.3.0
!pip install keras==2.4.3

In [ ]:
!git clone https://github.com/SherilK/keras-retinanet.git
!mkdir -p drive
!google-drive-ocamlfuse drive
%cd keras-retinanet/
!pip install . --user

!python3 setup.py build_ext --inplace

#Must create EMTD folder and download EMTD.zip to drive.  
Unzips EMTD zip file into the EMTD folder. 



In [ ]:
from google.colab import drive
drive.mount('/content/drive')


!unzip /content/drive/MyDrive/EMTD.zip -d /content/EMTD_ALL

In [ ]:
#TO CONTINUE ON TRAINING FROM PREVIOUS MODEL
!ln -s "/content/drive/MyDrive/Resnet_Models/resnet50_csv_75.h5" "/content/keras-retinanet/keras_retinanet/bin/snapshots"


#Rewrite Annotions for filenames with DSC<> and P<> to change jpg to JPG

In [ ]:
import pandas as pd
df = pd.read_csv('/content/EMTD_ALL/EMTD/Detection/GT39_train_3.csv', header=None)
df

,0,1,2,3,4,5
0,DSC-0134.JPG,1894,1022,1949,1078,Obstacles ahead
1,P1840410.JPG,1420,1881,1700,2200,Split way
2,P1840312.JPG,2257,1019,3476,2245,Keep left
3,P1840522.JPG,2387,1744,2420,1776,Minor road on left
4,P1840071.JPG,309,1329,605,1613,Speed limit 50
...,...,...,...,...,...,...
1882,P1840580.JPG,2438,712,2794,1107,No entry
1883,Screenshot (96).png,1407,527,1440,571,Crossroads to the left
1884,Screenshot (168).png,754,464,798,506,Crossroads to the right
1885,P1840068.JPG,3912,1549,4152,1786,Speed limit 50


In [ ]:

images = []
rows = df.shape[0]

for x in range(rows):
  i = df.iloc[x][0]
  if i in images:
    pass
  else:
    images.append(i)
len(images)

1094

In [ ]:
rows = df.shape[0]
df.iloc[0][0]
df

In [ ]:
#GET MIN SIDE#
df[[1,2]]
min = 9999
for xmin, ymin in zip(df[1], df[2]):
  #print(xmin, ymin)
  if xmin < ymin:
    if xmin < min:
      min = xmin
  elif ymin < xmin:
    if ymin < min:
      min = ymin
min

In [ ]:
#GET MAX SIDE#
max = 0
for xmax, ymax in zip(df[3], df[4]):
  #print(xmin, ymin)
  if xmax > ymax:
    if xmax > max:
      max = xmax
  elif ymax > xmax:
    if ymax > max:
      max = ymax
max

In [ ]:

for i in range(rows):
  file = df.iloc[i][0]
  if (file != 'P1860652.jpg') and (file !='P1860653.jpg') and (file !='P1860654.jpg') and (file !='P1860655.jpg') and (file !='P1860656.jpg'):
    if (file[0] == 'P') or (file[0] == 'D'):
      file = file.replace("jpg", "JPG")
      df.at[i, 0] = file
      #df.iloc[i][0] = file


#df.to_csv('/content/EMTD_ALL/EMTD/Detection/GT39.csv', index=False, header=False)

In [ ]:
#TRAIN/TEST SPLIT#
#df = df.drop(df.index[[0,1]])
#df.reset_index(drop=True, inplace=True)
df.to_csv('/content/EMTD_ALL/EMTD/Detection/GT39.csv', index=False, header=False)

In [ ]:
df
import os
print(len(os.listdir('/content/EMTD_ALL/EMTD/Detection')))
df

# Train Keras-RetinaNet using EMTD

In [ ]:
#%cd -
%cd keras-retinanet/
%cd keras_retinanet/
%cd bin/

/content/keras-retinanet
/content/keras-retinanet/keras_retinanet
/content/keras-retinanet/keras_retinanet/bin


In [ ]:
!ls

convert_model.py  debug.py  evaluate.py  __init__.py  train.py


In [ ]:
%cd - 

/content/keras-retinanet/keras_retinanet/bin


In [ ]:
!python train.py --epochs 4 --steps 1094 --batch-size 1 --config /content/Config/config.ini  --freeze-backbone --random-transform --snapshot-path /content/keras-retinanet/keras_retinanet/bin/snapshots  --snapshot /content/keras-retinanet/keras_retinanet/bin/snapshots/resnet50_csv_05.h5  csv /content/EMTD_ALL/EMTD/Detection/GT39_train_3.csv /content/EMTD_ALL/EMTD/Detection/labels_3.csv --val-annotations /content/EMTD_ALL/EMTD/Detection/GT39_val_1.csv 
#!python train.py --epochs 10 --steps 1094 --batch-size 1 --config /content/Config/config.ini  --weights /content/keras-retinanet/keras_retinanet/bin/snapshots/resnet50_csv_75.h5  --random-transform --freeze-backbone csv /content/EMTD_ALL/EMTD/Detection/GT39_train_3.csv /content/EMTD_ALL/EMTD/Detection/labels_3.csv --val-annotations /content/EMTD_ALL/EMTD/Detection/GT39_val_1.csv